In [0]:
!pip install tweepy
!pip install --upgrade azure-cognitiveservices-vision-face

In [0]:
import tweepy
import pandas as pd
import numpy as np
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
from PIL import Image
import requests
from io import BytesIO
import pandas as pd

In [0]:
twitter_keys = {
        'consumer_key':        '',
        'consumer_secret':     '',
        'access_token_key':    '',
        'access_token_secret': ''
    }

In [0]:
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth)

In [0]:
face_client = FaceClient('https://facedetecttweet.cognitiveservices.azure.com/', CognitiveServicesCredentials(''))

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Depression_Tweets_All.xlsx to Depression_Tweets_All.xlsx


In [0]:
import io
df = pd.read_excel(io.BytesIO(uploaded['Depression_Tweets_All.xlsx']), sheet_name="Type 2")

In [0]:
df.head()

,depression_type,tweet_type,Tweet,Twitter_handle,Tweet Date,Date of Diagnosis,Remarks
0,"Depression, Anxiety",2,1 year ago today I was diagnosed with anxiety ...,playboyzeeee,2019-04-11 03:01:11+00:00,NaN,NaN
1,Severe Depression,2,5 years ago today i was diagnosed with severe ...,grandefIynn,2019-04-16 13:42:06+00:00,NaN,NaN
2,"Depression, Anxiety",2,ONE YEAR AGO TODAY I'VE BEEN DIAGNOSED WITH DE...,enjenriquez,2019-04-24 03:08:02+00:00,NaN,NaN
3,Depression,2,"Exactly a year ago, I was diagnosed with depre...",charteredtax,2019-04-30 14:52:17+00:00,NaN,NaN
4,"Depression, Anxiety",2,A year ago today I had my first Anxiety attack...,jesuslovecoffee,2019-05-08 15:10:06+00:00,NaN,NaN


In [0]:
user_list = []
for user in df['Twitter_handle'].dropna():
    user_list.append(user)
print(user_list)

In [0]:
cols = ['Twitter_ID','Image URL','Age','Gender']
df_user = pd.DataFrame(columns=cols)
df_user.head()

In [0]:
image_url = []
row_index = 0
for user in user_list:
    try:
        tweets = api.get_user(user)
        imgURL = tweets.profile_image_url.replace('_normal','')
        
        df_user.loc[row_index,'Twitter_ID'] = user
        df_user.loc[row_index, 'Image URL'] = imgURL
        
        #response = requests.get(imgURL)
        #img = Image.open(BytesIO(response.content))
        
        #img_address = os.path.basename(img)
        detected_faces = face_client.face.detect_with_url(url=imgURL,
                                                          return_face_attributes=["age","gender","headPose","smile","facialHair","glasses","emotion",
                                                                                  "hair","makeup","occlusion","accessories","blur","exposure","noise"],
                                                          detection_model='detection_01',return_face_landmarks=True)
        if not detected_faces:
            df_user.loc[row_index,['Age', 'Gender']] = 'NA'
           # df_user.loc[row_index,'Gender'] = 'NA'
            row_index=row_index+1
        else:
            for face in detected_faces:
                df_user.loc[row_index, ['Age']] = face.face_attributes.age
                df_user.loc[row_index, ['Gender']] = face.face_attributes.gender.replace('Gender.','')
                row_index=row_index+1
    except:
        pass

In [0]:
df_user.head()

,Twitter_ID,Image URL,Age,Gender
0,playboyzeeee,http://pbs.twimg.com/profile_images/1265504389...,23,female
1,grandefIynn,http://pbs.twimg.com/profile_images/1265524410...,26,female
2,enjenriquez,http://pbs.twimg.com/profile_images/1262604701...,3,male
3,charteredtax,http://pbs.twimg.com/profile_images/1206459130...,63,male
4,jesuslovecoffee,http://pbs.twimg.com/profile_images/1255025818...,20,female


In [0]:
df_user.to_csv('Type2Users_profile.csv')
df_user.shape